In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shapefile as shp 

In [7]:
map_results_folder = '../map_results_onshore_2019' 

In [9]:
def get_elcc_list(region,year,capacity,technology,add_on=None):
    elcc_map_file = '%s/%s_%d_%d_MW_%s%s_results_map.csv' % (map_results_folder,region,year,capacity,technology,'' if add_on is None else '_'+add_on)
    elcc_df = pd.read_csv(elcc_map_file,index_col=0,header=0)
    elcc = (elcc_df.values).flatten()
    elcc = elcc[np.logical_not(np.isnan(elcc))]
    return elcc

In [39]:
def plot_line(ax, elcc_1, elcc_2, elcc_3):

    sns.set_theme(style="whitegrid")

    for i in range(len(elcc_1)):
        ax.plot([1,2,3],[elcc_1[i],elcc_2[i],elcc_3[i]],c='r',lw=.5)

    ax.set_xticks([1,2,3])

    ax.grid(False)
    ax.set_ylim(bottom=0)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    

    return ax

In [40]:

fig, axs = plt.subplots(5,2,dpi=240, figsize=(7,10),constrained_layout=True)

year = 2019

i = 0

for region in ['basin','california','mountains','northwest','southwest']:
    for technology in ['solar','wind']:
        ax = axs[i//2,i%2]

        elcc_100 = get_elcc_list(region,year,100,technology)
        elcc_500 = get_elcc_list(region,year,500,technology)
        elcc_2000 = get_elcc_list(region,year,2000,technology)
        
        ax = plot_line(ax,elcc_100,elcc_500,elcc_2000)
        ax.set_xticklabels(['100 MW','500 MW','2000 MW'])

        title = '%s %s' % (region.capitalize(),technology.capitalize())
        ax.set_title(title)

        i +=1


img_name = '../gallery/figure_4_generator_size_%d' % year
plt.savefig(img_name)  
plt.close()           


In [60]:
def plot_box(ax, region, generator_size, elcc):

    sns.set_theme(style="whitegrid")

    sns.boxplot(x=region, y=elcc, hue=generator_size,ax=ax)
    ax.grid(False)
    
    return ax

In [61]:

fig, axs = plt.subplots(2,1,dpi=240, figsize=(7,8),constrained_layout=True)

year = 2019
for technology in ['solar','wind']:
    ax = axs[0 if technology=='solar' else 1]

    region_arr = np.array([])
    elcc = np.array([])
    generator_size = np.array([])

    for region in ['basin','california','mountains','northwest','southwest']:
        
            elcc_100 = get_elcc_list(region,year,100,technology)
            elcc_500 = get_elcc_list(region,year,500,technology)
            elcc_2000 = get_elcc_list(region,year,2000,technology)
            
            generator_size = np.append(generator_size,np.array([np.ones(len(elcc_100))*100,np.ones(len(elcc_500))*500,np.ones(len(elcc_2000))*2000]).flatten())
            elcc = np.append(elcc, np.array([elcc_100,elcc_500,elcc_2000]).flatten())
            region_arr = np.append(region_arr, np.array([region.capitalize() for i in range(len(elcc_100)*3)]))

    ax = plot_box(ax,region_arr, generator_size, elcc)

    title = '%s' % (technology.capitalize())
    ax.set_title(title)      

img_name = '../gallery/figure_4_generator_size_box_%d' % year
plt.savefig(img_name)  
plt.close()